In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
tf.logging.set_verbosity(tf.logging.ERROR)

# Scalers
from sklearn.preprocessing import StandardScaler

from sklearn.compose import make_column_transformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import OneHotEncoder

from aif360.sklearn.preprocessing import ReweighingMeta
from aif360.sklearn.inprocessing import AdversarialDebiasing
from aif360.sklearn.postprocessing import CalibratedEqualizedOdds, PostProcessingMeta
from aif360.sklearn.datasets import fetch_compas
from aif360.sklearn.metrics import disparate_impact_ratio, average_odds_error, generalized_fpr
from aif360.sklearn.metrics import generalized_fnr, difference

In [3]:
X, y = fetch_compas()
X.head()

,,,sex,age,age_cat,race,juv_fel_count,juv_misd_count,juv_other_count,priors_count,c_charge_degree,c_charge_desc
id,sex,race,,,,,,,,,,
1,Male,Other,Male,69,Greater than 45,Other,0,0,0,0,F,Aggravated Assault w/Firearm
3,Male,African-American,Male,34,25 - 45,African-American,0,0,0,0,F,Felony Battery w/Prior Convict
4,Male,African-American,Male,24,Less than 25,African-American,0,0,1,4,F,Possession of Cocaine
7,Male,Other,Male,44,25 - 45,Other,0,0,0,0,M,Battery
8,Male,Caucasian,Male,41,25 - 45,Caucasian,0,0,0,14,F,Possession Burglary Tools


In [4]:
X.index = pd.MultiIndex.from_arrays(X.index.codes, names=X.index.names)
y.index = pd.MultiIndex.from_arrays(y.index.codes, names=y.index.names)

In [5]:
y = pd.Series(y.factorize(sort=True)[0], index=y.index)

In [6]:
(X_train, X_test,
 y_train, y_test) = train_test_split(X, y, train_size=0.7, random_state=1234567)

In [15]:
ohe = make_column_transformer((StandardScaler(), X_train.dtypes == 'numerical'), remainder='passthrough')
X_train  = pd.DataFrame(ohe.fit_transform(X_train), index=X_train.index)
X_test = pd.DataFrame(ohe.transform(X_test), index=X_test.index)

X_train.head()

,,,0,1,2,3,4,5,6,7,8,9
id,sex,race,,,,,,,,,,
2106,0,2,Male,22,Less than 25,Caucasian,0,0,0,0,M,DUI Level 0.15 Or Minor In Veh
5330,0,0,Male,39,25 - 45,African-American,0,0,0,1,F,Unlawful Conveyance of Fuel
5916,0,5,Male,31,25 - 45,Other,0,0,0,0,F,Pos Cannabis W/Intent Sel/Del
4158,1,2,Female,51,Greater than 45,Caucasian,0,0,0,0,F,Battery on Law Enforc Officer
4029,0,0,Male,21,Less than 25,African-American,0,0,0,1,F,Grand Theft in the 3rd Degree


In [ ]:
X.workclass.cat.remove_unused_categories(inplace=True)
pd.get_dummies(X).head()

In [ ]:
y_train.head()

In [ ]:
y_pred = LogisticRegression(solver='lbfgs').fit(X_train, y_train).predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
disparate_impact_ratio(y_test, y_pred, prot_attr='sex')